In [1]:
import yaml
from access import *

In [2]:
url_cam_given_account = "database-mm2503.cgrre17yxw11.eu-west-2.rds.amazonaws.com"
url_my_own_account = "testdatabase-mariadb.czjzatt1ot3b.eu-west-2.rds.amazonaws.com"
database_details = {"url": url_my_own_account, 
                    "port": 3306}
with open("credentials.yaml") as file:
  credentials = yaml.safe_load(file)
username = credentials["username"]
password = credentials["password"]
url = database_details["url"]

In [3]:
conn = create_connection(user=credentials["username"], 
                         password=credentials["password"], 
                         host=database_details["url"],
                         database="household_prices")
table_name = "pp_data"
sample_table_name = "pp_data_sample"

In [4]:
print(conn)
conn.ping()

In [5]:
cur = conn.cursor()
#cur.execute("USE `household_prices`")
cur.execute("show tables")
result = cur.fetchall()
print(f"tables: {result}")

tables: (('pp_data',), ('pp_data_sample',))


In [6]:
cur = conn.cursor()
cur.execute("SELECT COUNT(*) from pp_data_sample")
result = cur.fetchall()
print(f"count pp_data_sample: {result}")

count pp_data_sample: ((0,),)


In [4]:
df = select_top_to_df(conn, table_name)
df.head()

,transaction_unique_identifier,price,date_of_transfer,postcode,property_type,new_build_flag,tenure_type,primary_addressable_object_name,secondary_addressable_object_name,street,locality,town_city,district,county,ppd_category_type,record_status,db_id
0,{5BBE9CB3-6332-4EB0-9CD3-8737CEA4A65A},42000,1995-12-21,NE4 9DN,S,N,F,8,,MATFEN PLACE,FENHAM,NEWCASTLE UPON TYNE,NEWCASTLE UPON TYNE,TYNE AND WEAR,A,A,1
1,{20E2441A-0F16-49AB-97D4-8737E62A5D93},95000,1995-03-03,RM16 4UR,S,N,F,30,,HEATH ROAD,GRAYS,GRAYS,THURROCK,THURROCK,A,A,2
2,{D893EE64-4464-44B5-B01B-8E62403ED83C},74950,1995-10-03,CW10 9ES,D,Y,F,15,,SHROPSHIRE CLOSE,MIDDLEWICH,MIDDLEWICH,CONGLETON,CHESHIRE,A,A,3
3,{F9F753A8-E56A-4ECC-9927-8E626A471A92},43500,1995-11-14,TS23 3LA,S,N,F,19,,SLEDMERE CLOSE,BILLINGHAM,BILLINGHAM,STOCKTON-ON-TEES,STOCKTON-ON-TEES,A,A,4
4,{E166398A-A19E-470E-BB5A-83B4C254CF6D},63000,1995-09-08,CA25 5QH,S,N,F,8,,CROSSINGS CLOSE,CLEATOR MOOR,CLEATOR MOOR,COPELAND,CUMBRIA,A,A,5


In [8]:
drop_and_create_table_for_priced_paid_data(conn, "pp_data")
drop_and_create_table_for_priced_paid_data(conn, "pp_data_sample")

In [9]:
num_fetched_rows, num_uploaded_rows = upload_datasets(conn, "pp_data", "pp_data_sample", 1995, 1996)

fetching data from year: 1995, part: 1
uploading the data
fetching data from year: 1995, part: 2
uploading the data


In [10]:
print(num_fetched_rows)
print(num_uploaded_rows)

[398426, 398426]
[398426, 398426]


In [11]:
total_rows = custom_simple_select(conn, "pp_data")
sample_rows = custom_simple_select(conn, "pp_data_sample")
print(f"total_rows: {total_rows}, sample_rows: {sample_rows}")

total_rows: ((796852,),), sample_rows: ((7968,),)


In [12]:
conn.commit()
conn.close()

In [20]:
num_fetched_rows, num_uploaded_rows = upload_2022_dataset(conn)

In [21]:
print(num_uploaded_rows, num_fetched_rows)

456603 456603
